In [1]:
!pip install torch transformers accelerate sentence-transformers langchain faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 13.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-3.2-1B-Instruct"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("Llama 3.2-1B model loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Llama 3.2-1B model loaded successfully!


In [32]:
import torch

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model and tokenizer
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Move input to GPU
    with torch.no_grad():
        output = model.generate(**inputs, max_length=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test the model
#prompt = "Explain neural networks."
#prompt = "Recent advances in AI?"
prompt = "What is deep learning?"
response = generate_text(prompt)
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is deep learning? Deep learning is a type of artificial intelligence (AI) that uses complex algorithms and statistical models to analyze and interpret data. It is based on the idea of representing data as a set of interconnected nodes or "neurons" that process and transmit information.

Deep learning is different from other types of AI, such as rule-based systems, which use pre-defined rules to make decisions. Instead, deep learning uses a set of artificial neural networks, which are inspired by the structure and function of the human brain. These networks are composed of multiple layers, each with its own set of nodes or "neurons" that process and transmit information.

Deep learning has many applications, including image and speech recognition, natural language processing, and predictive modeling. It is also used in a wide range of industries, such as finance, healthcare, and transportation.

Some of the key characteristics of deep learning include:

1. **Complexity**: Deep lear

In [15]:
!pip install wikipedia-api


In [14]:
import wikipediaapi

# Définir un User-Agent valide
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent="MyWikipediaScraper/1.0 (chahedjouini59@gmail.com)"
)

topics = ["Artificial Intelligence", "Machine Learning", "Deep Learning", "Neural Networks"]

# Extraire le texte des articles
documents = []
for topic in topics:
    page = wiki_wiki.page(topic)
    if page.exists():
        documents.append(page.text)

print(f"Collected {len(documents)} Wikipedia articles.")


Collected 4 Wikipedia articles.


In [21]:
!pip install langchain faiss-cpu transformers sentence-transformers



In [22]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [23]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Use sentence-transformers for text embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-23-92e48c744cd2>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_text("\n".join(documents))

# Store in FAISS for retrieval
db = FAISS.from_texts(docs, embedding_model)
print("Knowledge Base Created!")





Knowledge Base Created!


In [30]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Define Retriever
retriever = db.as_retriever()

# Define Model Pipeline
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Load a QA chain
chain_type_kwargs = {"verbose": True}
combine_documents_chain = load_qa_chain(llm, chain_type="stuff", **chain_type_kwargs)

# Initialize RetrievalQA with the chain
rag_model = RetrievalQA(retriever=retriever, combine_documents_chain=combine_documents_chain)

Device set to use cuda:0
<ipython-input-30-a1f776cb772c>:14: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  combine_documents_chain = load_qa_chain(llm, chain_type="stuff", **chain_type_kwargs)


In [37]:
# Charger ou initialiser le modèle RAG
def test_models(query):
    print(f"\nQuery: {query}")

     # With RAG
    print("With RAG:", rag_model.run(query))

queries = [
    "What is deep learning?",
    "Explain neural networks.",
    "Recent advances in AI?"
]

for query in queries:
    test_models(query)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: What is deep learning?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

International Machine Learning Society
mloss is an academic database of open-source machine learning software.
Deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.
Some common deep learning network architectures include fully 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.

> Finished chain.
With RAG: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

International Machine Learning Society
mloss is an academic database of open-source machine learning software.
Deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.
Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.

> Finished chain.
With RAG: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In machine learning
In machine learning, a neural network is an artificial mathematical model used to approximate nonlinear functions. While early artificial neural networks were physical machines, today they are almost always implemented in software.
Neurons in an artificial neural network are usually arranged into layers, with information passing from the first layer (the input layer) through one or more intermediate layers (hidden layers) to the final layer (the output layer).
The "signal" input to each neuron is a number, specifically a linear combination of the outputs of the connected neurons in the previous layer. The signal each neuron outputs is calculated from this number, according to its activation function. The behavior of the network depends on the strengths (or w